In [1]:
import pandas as pd
import pyreadstat
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

import econtools as econ
import econtools.metrics as mt
import math


In [39]:
# 3차 
df_HH3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_household.sav")

#소득 [F2H04001]
df_HH3["F2H04001"].describe()
df_HH3 = df_HH3.rename(columns = {"F2H04001": "income"})
df_HH3 = df_HH3[df_HH3["income"]>0]
df_HH3["year"] = 2003
df_HH3 = df_HH3[['BYHID','income', 'year']]

In [40]:
# 4차
df_HH4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_household.sav")

#소득 F3H08001
df_HH4["F3H08001"].describe()
df_HH4 = df_HH4.rename(columns = {"F3H08001": "income"})
df_HH4 = df_HH4[df_HH4["income"]>0]
df_HH4["year"] = 2004
df_HH4 = df_HH4[['BYHID','income', 'year']]

In [41]:
df_HH = pd.concat([df_HH3, df_HH4])
len(df_HH)

4057

<h2>Students</h2>

In [17]:
# 3차_학생 고등학교 일반계 , 전문계

df_st3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_highschool_st.sav")
df_st_v3, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/3차년도(2006) DATA/중학교 3학년생 코호트/f2_m_vocationalhighschool_st.sav")
df3 = pd.concat([df_st3, df_st_v3])
df3["year"] = 2003
df3 = df3.rename(columns={'F2S01003':'schooltype'})
df3 = df3.rename(columns={'F2S01002':'region'})
df3.loc[df3['BYSID'].isin(df_st_v3['BYSID'])==True, 'schooltype'] = 0

# school type is not specified yet (in survey)

In [18]:
# 4차_학생 고등학교 일반계, 전문계

df_st4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_highschool_st.sav")
df_st_v4, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/4차년도(2007) DATA/중학교 3학년생 코호트/f3_m_vocationalhighschool_st.sav")

df4 = pd.concat([df_st4, df_st_v4])
df4["year"] = 2004
df4 = df4.rename(columns={'F3S01003':'schooltype'})
df4 = df4.rename(columns={'F3S01002':'region'})

df4.loc[df4['BYSID'].isin(df_st_v4['BYSID'])==True, 'schooltype'] = 0

In [19]:
df4['schooltype'].isna().value_counts() #1no nan

False    2976
Name: schooltype, dtype: int64

In [25]:
# Allocate df3 schooltype for students who is not in df4
st_not_specified = df3.loc[df3['BYSID'].isin(df4['BYSID'])==False, 'BYSID'].to_list()
len(st_not_specified) # 114 students

#asking specific major
df3['F2S01023'].value_counts()
df3.loc[df3['BYSID'].isin(st_not_specified) == True, 'F2S01023'].value_counts() # 70 are public school (-5)
# english 1, science 1, actor 1
#df3.loc[(df3['BYSID'].isin(st_not_specified) == True) & (df3['F2S01004'] == 4.0), 'BYSID'] : 0 -> 70 are public schools
df3.loc[df3['F2S01023'] == -5, 'schooltype'] = 7.0
df3.loc[df3['F2S01023'] == '영어', 'schooltype'] = 1.0
df3.loc[df3['F2S01023'] == '과학계', 'schooltype'] = 2.0
df3.loc[df3['F2S01023'] == '연극', 'schooltype'] = 3.0

# df3 school type was not specified
# Allocate df3 schooltype by schooltype in df4

for i in df3['BYSID']:
    if i in df4['BYSID']:
        df3.loc[df3['BYSID'] == i, 'schooltype' ] = df4.loc[df4['BYSID'] ==i, 'schooltype']

In [37]:
df3 = df3[df3['schooltype']>0]
df4 = df4[df4['schooltype']>0]

df = pd.concat([df3, df4])

In [38]:
len(df)

2512

In [45]:
# 남 1, 여 0
df.loc[df['GENDER']==2.0, 'GENDER'] =0

<h2>Main Dataset</h2>

In [43]:
df = pd.merge(df, df_HH)
#df3 does not have information about teacehrs #mighbe use 1차년도 teacher (2차년도-고1-teacher도 없음)

In [44]:
len(df) # parents no response = -500

2069

In [46]:
# income rank 03 
a = df.loc[df['year'] == 2003, "income"]

df["income_rank_p"] = np.nan
df.loc[ df['year'] == 2003 , "income_rank_p"] = a.rank(method = 'dense') 
df.loc[ df['year'] == 2003 , "income_rank_p"] = df.loc[ df['year'] == 2003 , "income_rank_p"]/ df.loc[ df['year'] == 2003 , "income_rank_p"].max()

# income rank 04
a = df.loc[df['year'] == 2004, "income"]

df.loc[ df['year'] == 2004 , "income_rank_p"] = a.rank(method = 'dense') 
df.loc[ df['year'] == 2004 , "income_rank_p"] = df.loc[ df['year'] == 2004 , "income_rank_p"]/ df.loc[ df['year'] == 2004 , "income_rank_p"].max()

In [48]:
df["income_rank_p"].isna().value_counts() # all valid

False    2069
Name: income_rank_p, dtype: int64

<h2>Achievement Goal</h2>

In [49]:
# achievement goal
# 3차 achievement goals = reference column
df = df.rename(columns = {'F2S01071': 'learning', 'F2S01072': 'betterperson', 'F2S01073': 'betterjob', 'F2S01074': 'futurework', 'F2S01075': 'forsociety', 'F2S01076': 'favorablejudg', 'F2S01077': 'popular', 'F2S01078': 'competition', 'F2S01079': 'forced', 'F2S01080': 'negativejudg', 'F2S01081': 'penalty'})

In [50]:
# 4차 90은 money인데 3차에는 없음
df.loc[df['year'] == 2004, 'learning'] = df['F3S01087']
df.loc[df['year'] == 2004, 'betterperson'] = df['F3S01088']
df.loc[df['year'] == 2004, 'betterjob'] = df['F3S01089']
df.loc[df['year'] == 2004, 'futurework'] = df['F3S01091']
df.loc[df['year'] == 2004, 'forsociety'] = df['F3S01092']
df.loc[df['year'] == 2004, 'favorablejudg'] = df['F3S01093']
df.loc[df['year'] == 2004, 'popular'] = df['F3S01094']
df.loc[df['year'] == 2004, 'competition'] = df['F3S01095']
df.loc[df['year'] == 2004, 'forced'] = df['F3S01096']
df.loc[df['year'] == 2004, 'negativejudg'] = df['F3S01097']
df.loc[df['year'] == 2004, 'penalty'] = df['F3S01098']

In [51]:
# drop achievement goal missing values

goal_list = ['learning', 'betterperson', 'betterjob', 'futurework', 'forsociety', 'favorablejudg',  'popular',  'competition', 'forced', 'negativejudg',  'penalty']
for i in goal_list:
    df = df[df[i]>0]
    
# drop previous columns
drop_list = []

for i in range(87, 99):
    df = df.drop('F3S010'+str(i), axis = 1)

In [29]:
pyreadstat.write_sav(df, 'df.sav')

<h3>School Type</h3>

학교 별 소득 평균에 따라 값 부여

In [ ]:
df.groupby('schooltype')['income'].mean()
# 6. 대안학교 < 0. 전문계 (252) < 5. 종합고 일반계 < 7. 일반고 < 1. 외국어고 < 2. 과학고 <3. 예체능 < 4. 자립형 사립고

df['st_pi'] = np.nan
df.loc[df['schooltype'] == 6, 'st_pi'] =0
df.loc[df['schooltype'] == 0, 'st_pi'] =1
df.loc[df['schooltype'] == 5, 'st_pi'] =2
df.loc[df['schooltype'] == 7, 'st_pi'] =3
df.loc[df['schooltype'] == 1, 'st_pi'] =4
df.loc[df['schooltype'] == 2, 'st_pi'] =5
df.loc[df['schooltype'] == 3, 'st_pi'] =6
df.loc[df['schooltype'] == 4, 'st_pi'] =7

<h2> HH1 - parents' information </h2>

In [57]:
df_HH1, meta = pyreadstat.read_sav("C:/Users/thxkn/OneDrive/바탕 화면/CV/박사지원서류/WS_HJ/KEEP/KEEPData_1-12Wave_Ver12/DATA(수능, 직업력, 가중치, 패널조사현황 자료 포함)/1-12차년도 DATA(SPSS)/1차년도(2004) DATA/중학교 3학년생 코호트/by_m_household.sav")